In [94]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [95]:
# load data
data = pd.read_excel("감자 (1).xlsx")
production = pd.read_excel("생산량 (1).xlsx")


# merge data by DATE, if DATE is not in production, replace it with NaN
data = data.merge(production, on='DATE', how='left')

In [96]:
# load weathers data
weathers_1 = pd.read_excel("기상.xlsx")
weathers_2 = pd.read_excel("농업 날씨.xlsx")

In [97]:
# extract unique columns in weathers_1 for 1st column
weathers_1_columns = []
for column in weathers_1.columns:
    if column not in weathers_1_columns:
        weathers_1_columns.append(column)


weathers_1[weathers_1_columns[0]]
location_names = []
for row in weathers_1[weathers_1_columns[0]]:
    if row not in location_names:
        location_names.append(row)  
        
# changes values name to English in weathers_1 for 1st column
weathers_1[weathers_1_columns[0]] = weathers_1[weathers_1_columns[0]].replace(location_names, ["choeone", "suone", "andong", "geonju", "sunjeon"])

# changes column name to English in weathers_1
weathers_1.columns = ["location", "DATE", "rainfall", "total_sunshine", "total_solar", "snowfall"]

# average weathers_1 by DATE, if DATE is not in production, replace it with 0
weathers_1 = weathers_1.groupby('DATE').mean().reset_index()
weathers_1 = weathers_1.fillna(0)

In [98]:
weathers_1

,DATE,rainfall,total_sunshine,total_solar,snowfall
0,2000-01-01,0.000000,2.575,4.363333,0.0
1,2000-01-02,6.950000,1.300,2.280000,0.0
2,2000-01-03,0.000000,7.950,7.026667,0.0
3,2000-01-04,0.000000,5.325,5.253333,0.0
4,2000-01-05,11.150000,0.075,1.466667,9.5
...,...,...,...,...,...
9071,2024-11-01,15.800000,0.480,3.490000,0.0
9072,2024-11-02,6.950000,6.040,11.625000,0.0
9073,2024-11-03,0.000000,7.960,13.005000,0.0
9074,2024-11-04,0.466667,5.540,9.970000,0.0


In [99]:
# same process for weathers_2
weathers_2_columns = []
for column in weathers_2.columns:
    if column not in weathers_2_columns:
        weathers_2_columns.append(column)

weathers_2[weathers_2_columns[0]]
location_names = []
for row in weathers_2[weathers_2_columns[0]]:
    if row not in location_names:
        location_names.append(row)

weathers_2[weathers_2_columns[0]] = weathers_2[weathers_2_columns[0]].replace(location_names, ["suone", "choeone", "andong", "geonju", "sunjeon"])
weathers_2.columns = ["location", "DATE", "average_humidity", "30cm_soil_temperature", "average_temperature", "average_wind", "total_sunshine"," total_snowfall"]

weathers_2 = weathers_2.groupby('DATE').mean().reset_index()
weathers_2 = weathers_2.fillna(0)


In [100]:
# total weathers dataframes
weathers_set_1 = weathers_1[["DATE", "rainfall", "snowfall", "total_sunshine", "total_solar"]]
weathers_set_2 = weathers_2[["DATE", "average_humidity", "30cm_soil_temperature", "average_temperature",
                            "average_wind"]]

# create a new dataframe with weathers_set_1 and weathers_set_2 by DATE
weathers = weathers_set_1.merge(weathers_set_2, on='DATE', how='left')


#### 날씨데이터에서 결측치는 0으로 대체

In [101]:
# merge data with weathers
data = data.merge(weathers, on='DATE', how='left')

In [102]:
# rename columns    
data.columns = ["DATE", "avg_price", "pre_day_price", "pre_year_price", "pre_7day_price", "production"] + list(data.columns[6:])


In [ ]:
# add data
# load data 
input_labor = pd.read_excel("노동투입.xlsx")
income = pd.read_excel("농가소득.xlsx")
inflation = pd.read_excel("물가 지수.xlsx")
oil = pd.read_excel("유가.xlsx")
cultivation_area = pd.read_excel("재배 면적.xlsx")

In [106]:
# drop columns [2,3,5,6] in input_labor
input_labor = pd.read_excel("노동투입.xlsx")
input_labor = input_labor.drop(input_labor.columns[[2,3,5,6]], axis=1)

# rename columns
input_labor.columns = ["DATE", "spring_production", "fall_production"]

# drop 1st row in input_labor
input_labor = input_labor.drop([0])

# change yearly data to daily data when 2003-01-01 ~ 2022-12-31 
input_labor = input_labor.loc[input_labor.index.repeat(365)].reset_index(drop=True)

num_rows = len(input_labor)
input_labor["DATE"] = pd.date_range(start='2003-01-01', periods=num_rows, freq="D")

# extract february ~ August data from input_labor
input_labor["fall_production"][(input_labor['DATE'].dt.month >= 2) & (input_labor['DATE'].dt.month <= 8)] = input_labor["spring_production"][(input_labor['DATE'].dt.month >= 2) & (input_labor['DATE'].dt.month <= 8)]

# drop spring_production column
input_labor = input_labor.drop(["spring_production"], axis=1)

# change column name
input_labor.columns = ["DATE", "input_labor"]

input_labor

C:\Users\82108\AppData\Local\Temp\ipykernel_10664\4108076389.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_labor["fall_production"][(input_labor['DATE'].dt.month >= 2) & (input_labor['DATE'].dt.month <= 8)] = input_labor["spring_production"][(input_labor['DATE'].dt.month >= 2) & (input_labor['DATE'].dt.month <= 8)]


,DATE,input_labor
0,2003-01-01,72.8
1,2003-01-02,72.8
2,2003-01-03,72.8
3,2003-01-04,72.8
4,2003-01-05,72.8
...,...,...
7295,2022-12-22,45
7296,2022-12-23,45
7297,2022-12-24,45
7298,2022-12-25,45


In [111]:
# smme process for income for 2002~2023
income = pd.read_excel("농가소득.xlsx")

# column name change
income.columns = ["DATE", "spring_income", "fall_income"]

# daily data
income = income.loc[income.index.repeat(365)].reset_index(drop=True)

num_rows = len(income)
income["DATE"] = pd.date_range(start='2002-01-01', periods=num_rows, freq="D")



In [107]:
# same process for cultivation_area
cultivation_area = pd.read_excel("재배 면적.xlsx")

cultivation_area["cultivation_area"] = cultivation_area.sum(axis=1)

# drop columns
cultivation_area = cultivation_area.drop(cultivation_area.columns[[1,2,3,4,5,6]], axis=1)

# columns name
cultivation_area.columns = ["DATE", "cultivation_area"]

# change to daily data when 2000-01-01 ~ 2023-12-31
cultivation_area = cultivation_area.loc[cultivation_area.index.repeat(365)].reset_index(drop=True)

num_rows = len(cultivation_area)

cultivation_area["DATE"] = pd.date_range(start='2000-01-01', periods=num_rows, freq="D")


In [108]:
# oil data
oil = pd.read_excel("유가.xlsx")

# columns name
oil.columns = ["DATE", "oil"]


In [109]:
inflation = pd.read_excel("물가 지수.xlsx")
inflation.columns = ["DATE", "cost of living", "food index", "CPI", "fresh index"]

# Convert the "DATE" column to datetime format
inflation["DATE"] = pd.to_datetime(inflation["DATE"], errors='coerce')

# Function to expand monthly data to daily data
def expand_monthly_to_daily(df):
    daily_data = pd.DataFrame()
    for index, row in df.iterrows():
        # Generate daily dates for the month
        daily_dates = pd.date_range(start=row['DATE'].replace(day=1), periods=row['DATE'].days_in_month, freq='D')
        # Repeat the values for each day in the month
        daily_values = {col: [row[col]] * len(daily_dates) for col in df.columns if col != 'DATE'}
        daily_values['DATE'] = daily_dates
        # Create a temporary DataFrame for the daily data
        temp_df = pd.DataFrame(daily_values)
        # Append to the daily_data DataFrame
        daily_data = pd.concat([daily_data, temp_df], ignore_index=True)
    return daily_data

# Expand the monthly data to daily data
daily_inflation = expand_monthly_to_daily(inflation)

# Generate the continuous date range from 2000-01-01 to 2024-10-31
continuous_dates = pd.date_range(start='2000-01-01', end='2024-10-31', freq='D')

# Ensure the length of the continuous date range matches the length of the daily_inflation DataFrame
if len(continuous_dates) > len(daily_inflation):
    # Extend the daily_inflation DataFrame
    additional_rows = len(continuous_dates) - len(daily_inflation)
    last_row = daily_inflation.iloc[-1]
    additional_data = pd.DataFrame([last_row] * additional_rows)
    daily_inflation = pd.concat([daily_inflation, additional_data], ignore_index=True)
elif len(continuous_dates) < len(daily_inflation):
    # Truncate the daily_inflation DataFrame
    daily_inflation = daily_inflation.iloc[:len(continuous_dates)]

daily_inflation["DATE"] = continuous_dates

# relocation of columns
daily_inflation = daily_inflation[["DATE", "cost of living", "food index", "CPI", "fresh index"]]

In [112]:
# total merge data with oil, income, cultivation_area, input_labor
data = data.merge(oil, on='DATE', how='left')
data = data.merge(income, on='DATE', how='left')
data = data.merge(cultivation_area, on='DATE', how='left')
data = data.merge(input_labor, on='DATE', how='left')
data = data.merge(daily_inflation, on='DATE', how='left')


In [92]:
data.columns

Index(['DATE', 'avg_price', 'pre_day_price', 'pre_year_price',
       'pre_7day_price', 'production_x', 'rainfall', 'snowfall',
       'total_sunshine', 'total_solar', 'average_humidity',
       '30cm_soil_temperature', 'average_temperature', 'average_wind', 'oil_x',
       'spring_income_x', 'fall_income_x', 'cultivation_area_x',
       'fall_production', 'cost of living_x', 'food index_x', 'CPI_x',
       'fresh index_x', 'oil_y', 'spring_income_y', 'fall_income_y',
       'cultivation_area_y', 'production_y', 'cost of living_y',
       'food index_y', 'CPI_y', 'fresh index_y'],
      dtype='object')

In [ ]:
test = data.copy()

# replace 0 with NaN
test["avg_price"] = test["avg_price"].replace(0, np.nan)

# replace 0 with linear interpolation
test["avg_price"] = test["avg_price"].interpolate(method='linear')

# replace NaN with ffill
test["pre_day_price"] = test["pre_day_price"].replace(0, np.nan)
test["pre_day_price"] = test["pre_day_price"].fillna(method='ffill')

# replace NaN with ffill
test["pre_year_price"] = test["pre_year_price"].replace(0, np.nan)
test["pre_year_price"] = test["pre_year_price"].fillna(method='ffill')

# drop 0 in pre_7day_price
test = test[test["pre_7day_price"] != 0]

# replace NaN with ffill
test['oil_y'] = test["oil_y"].fillna(method='ffill')


In [117]:
test.to_excel("core_data.xlsx", index=False)